In [ ]:
%defaultDatasource jdbc:h2:mem:db

# Importando Dados e Criando Tabelas SQL

In [ ]:
DROP TABLE IF EXISTS Estado;
DROP TABLE IF EXISTS Cidade;
DROP TABLE IF EXISTS Aeroporto;
DROP TABLE IF EXISTS Periodo;
DROP TABLE IF EXISTS Rota;
DROP TABLE IF EXISTS Voo;
DROP TABLE IF EXISTS Casos;

In [ ]:
CREATE TABLE Estado
(
    UF VARCHAR(2) NOT NULL,
    Nome VARCHAR(40) NOT NULL,
    PRIMARY KEY(UF)
)
AS SELECT UF,Nome FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/estado.csv');

In [ ]:
CREATE TABLE Cidade
(
  Estado VARCHAR(2) NOT NULL,
  Nome VARCHAR(40) NOT NULL,
  CONSTRAINT PK_Cidade PRIMARY KEY (Nome,Estado),
  FOREIGN KEY(Estado) REFERENCES Estado(UF)
) 
AS SELECT Estado,Nome FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/cidadeFINAL.csv');

In [ ]:
CREATE TABLE Aeroporto
(
  Sigla VARCHAR(4) NOT NULL,
  Descricao VARCHAR(80) NOT NULL,
  Cidade VARCHAR(40) NOT NULL,
  PRIMARY KEY(Sigla)
) AS SELECT Sigla,Descricao,Cidade FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/aeroportoFINAL.csv');

In [ ]:
CREATE TABLE Periodo
(
  Id INT NOT NULL,
  Semana INT NOT NULL,
  Ano INT NOT NULL,
  PRIMARY KEY(Id,Semana,Ano)
) AS SELECT Id,Semana,Ano FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/periodo.csv');

In [ ]:
CREATE TABLE Rota
(
  Id INT NOT NULL,
  Origem VARCHAR(4) NOT NULL,
  Destino VARCHAR(4) NOT NULL,
  VoosTotais INT NOT NULL,
  PRIMARY KEY(Id),
  FOREIGN KEY(Origem) REFERENCES Aeroporto(Sigla),
  FOREIGN KEY(Destino) REFERENCES Aeroporto(Sigla)
) AS SELECT Id,Origem,Destino,VoosTotais FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/rota.csv')

In [ ]:
CREATE TABLE Voo
(
  Rota INT NOT NULL,
  Periodo INT NOT NULL,
  Quantidade INT NOT NULL,
  CONSTRAINT PK_Rota PRIMARY KEY (Rota,Periodo),
  FOREIGN KEY(Rota) REFERENCES Rota(Id),
  FOREIGN KEY(Periodo) REFERENCES Periodo(Id)
) AS SELECT Rota,Periodo,Quantidade FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/voo.csv');

In [ ]:
CREATE TABLE Casos
(
  Estado VARCHAR(40) NOT NULL,
  Periodo INT NOT NULL,
  NumCasos INT NOT NULL,
  CONSTRAINT PK_Casos PRIMARY KEY (Estado,Periodo),
  FOREIGN KEY(Periodo) REFERENCES Periodo(Id),
  FOREIGN KEY(Estado) REFERENCES Estado(Nome)
) AS SELECT Estado,Periodo,NumCasos FROM CSVREAD('https://raw.githubusercontent.com/Desnord/ProjetoFinalMC536/main/stage04/data/processed/casosFINAL.csv');

# total de casos de gripe por estado, entre 2010 e 2019

In [ ]:
select Estado, SUM(NumCasos) TotalDeCasos
from Casos
group by Estado;

# todos os aeroportos de algum estado (no caso SP)

In [ ]:
select a.sigla, c.Estado
from Aeroporto a, Cidade c
where a.Cidade = c.Nome and c.Estado = 'SP';

# todas as rotas de algum estado (no caso SP)

In [ ]:
select r.Id, a.sigla, c.Estado
from Rota r, Aeroporto a, Cidade c
where a.Cidade = c.Nome and c.Estado = 'SP' and r.Destino = a.Sigla;

# total de voos chegando em algum estado, por cada periodo

In [110]:
select c.Estado, v.Periodo, SUM(v.Quantidade) total
from Rota r, Aeroporto a, Cidade c, Voo v
where v.Rota = r.Id and r.Destino = a.Sigla and a.Cidade = c.Nome
group by v.Periodo,c.Estado;